#Работа с СУБД SQLite

In [1]:
import sqlite3 as sl
#sqlite3.api level
#sqlite3.sqlite_version

In [2]:
db1=sl.connect('3_5.db')

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
crs=db1.cursor()
crs.execute("""CREATE TABLE doctor(doctor_id serial NOT NULL PRIMARY KEY,
	surname varchar(50) NOT NULL,
	firstname varchar(50) NOT NULL,
	patronymic varchar(20) NULL,
	dep_id int NOT NULL,
	spec_id int NOT NULL)""")
sql_insert="""INSERT INTO doctor VALUES (1,'Федоров', 'Илья', 'Васильевич', 1, 1),(2, 'Мозгунова', 'Светлана', 'Яковлевна', 2, 3), (3, 'Крапивова', 'Екатерина', 'Игоревна', 3, 2)"""

try:
    crs.execute(sql_insert)
except sl.DatabaseError as err:
    print('Error: ', err)
else:
    db1.commit()
    print('Зaпpoc успешно выполнен')

crs.close()

Зaпpoc успешно выполнен


In [4]:
crs=db1.cursor()

crs.executescript("""CREATE TABLE deps (dep_id serial NOT NULL PRIMARY KEY, name varchar(100) NOT NULL);
CREATE TABLE spec(spec_id serial NOT NULL PRIMARY KEY, name varchar(100) NOT NULL);
INSERT INTO deps VALUES (1, 'Кардиологическое отделение'), (2, 'Пульмонологическое отделение'), (3, 'Гастроэнтерологическое отделение');
INSERT INTO spec  VALUES (1, 'Кардиолог'), (2, 'Пульмонолог'), (3, 'Гастроэнтеролог'),(4, 'Врач функциональной диагностики');""")
db1.commit()

In [5]:
crs=db1.cursor()
print('\nПросмотр всех записей таблицы doctor')
for r in crs.execute('SELECT * FROM doctor ORDER BY doctor_id '):
  print(r)


Просмотр всех записей таблицы doctor
(1, 'Федоров', 'Илья', 'Васильевич', 1, 1)
(2, 'Мозгунова', 'Светлана', 'Яковлевна', 2, 3)
(3, 'Крапивова', 'Екатерина', 'Игоревна', 3, 2)


In [6]:
crs=db1.cursor()
st=[(4,'Смирнов', 'Алексей', 'Михайлович', 1, 2),(5,'Арутян', 'Карен', 'Альбертович', 2, 3),(6,'Фаткуллин', 'Фарид', 'Ильдарович', 3, 2)]

try:
    crs.executemany('INSERT INTO doctor VALUES (?,?,?,?,?,?)',st)
except sqlite3.DatabaseError as err:
    print('Error: ', err)
    print('Последний идентификатор вставленной строки: ', crs.lastrowid)
else:
    db1.commit()
    print('Зaпpoc успешно выполнен')
crs.close()

Зaпpoc успешно выполнен


In [7]:
r=[]
crs=db1.cursor()
c=crs.execute('SELECT * FROM deps ORDER BY name')
for i in c:
  r.append(i)
r

[(3, 'Гастроэнтерологическое отделение'),
 (1, 'Кардиологическое отделение'),
 (2, 'Пульмонологическое отделение')]

In [8]:
crs.close()

Выполнение запросов к таблицам БД

In [9]:
import sqlite3
db1=sqlite3.connect('3_5.db')
crs=db1.cursor()

#получение результата запроса при помощи fetchall
sql='SELECT * FROM spec WHERE spec_id=?'
crs.execute(sql,[2])
print('Результат запроса 1 -> ',crs.fetchall())

#получение результата запроса с опцией LIKE
sql_1='SELECT * FROM doctor WHERE surname LIKE "Ф%"'
crs.execute(sql_1)
print('Результат запроса 2 -> ',crs.fetchall())

#получение результата запроса с соединением 2-х таблиц
sql_2='select doctor.surname, doctor.firstname, doctor. patronymic, deps.name, spec.name from deps inner join (spec inner join doctor ON spec.spec_id=doctor.spec_id) ON deps.dep_id=doctor.dep_id'
crs.execute(sql_2)
print('Результат запроса 3 -> ')
res=crs.fetchall()
for r in res:
  print(r,end='\n')
crs.close()

Результат запроса 1 ->  [(2, 'Пульмонолог')]
Результат запроса 2 ->  [(1, 'Федоров', 'Илья', 'Васильевич', 1, 1), (6, 'Фаткуллин', 'Фарид', 'Ильдарович', 3, 2)]
Результат запроса 3 -> 
('Федоров', 'Илья', 'Васильевич', 'Кардиологическое отделение', 'Кардиолог')
('Мозгунова', 'Светлана', 'Яковлевна', 'Пульмонологическое отделение', 'Гастроэнтеролог')
('Крапивова', 'Екатерина', 'Игоревна', 'Гастроэнтерологическое отделение', 'Пульмонолог')
('Смирнов', 'Алексей', 'Михайлович', 'Кардиологическое отделение', 'Пульмонолог')
('Арутян', 'Карен', 'Альбертович', 'Пульмонологическое отделение', 'Гастроэнтеролог')
('Фаткуллин', 'Фарид', 'Ильдарович', 'Гастроэнтерологическое отделение', 'Пульмонолог')


Работа с библиотекой Pandas

In [10]:
import pandas as pd
import sqlite3

db3=sqlite3.connect('3_5.db')
#df=pd.read_sql('CREATE TABLE patients (id integer PRIMARY KEY, FIO text, DR text)',db3)
#df=pd.read_sql('INSERT INTO patients(id,FIO, DR) VALUES (22,"Игнатьева А.Л.","2003")',db3)
df=pd.DataFrame([[22,'Игнатьева А.Л.','2003'],[3,'Плохих М.А.','2002'],[4,'Сергеев А.Т.','2002'],[5,'Андреев И.П.','2004'],[6,'Перовский И.В.','2004']],columns=['id','FIO','DR'])
df.to_sql('patients',db3,if_exists='replace')
df=pd.read_sql_query('SELECT * FROM patients limit 5;',db3)
df

,index,id,FIO,DR
0,0,22,Игнатьева А.Л.,2003
1,1,3,Плохих М.А.,2002
2,2,4,Сергеев А.Т.,2002
3,3,5,Андреев И.П.,2004
4,4,6,Перовский И.В.,2004


In [11]:
cur_2=db3.cursor()
cur_2.execute('ALTER TABLE patients ADD COLUMN gender nchar(3);');

In [12]:
table_name='patients'
pd.read_sql_query('select * from '+table_name+';',db3)

,index,id,FIO,DR,gender
0,0,22,Игнатьева А.Л.,2003,None
1,1,3,Плохих М.А.,2002,None
2,2,4,Сергеев А.Т.,2002,None
3,3,5,Андреев И.П.,2004,None
4,4,6,Перовский И.В.,2004,None


In [13]:
df=pd.read_sql('SELECT * FROM  patients',db3)
df['gender']=None
df.to_sql('patients',db3,if_exists='replace')

5

In [16]:
from datetime import datetime
df=pd.DataFrame([[1,'Sergeev BB','Пульмонологическое отделение',datetime(2023,11,29,12,20), datetime(2023,12,29,13,55)]],
                columns=['id','FIO_prep','subject','Lec_beg','Lec_end'])
df.to_sql('Report1',db3,if_exists='replace')
pd.read_sql_query('select * from Report1;',db3)
table_name='Report1'
#crs.execute('alter table Report1 add column room integer')
pd.read_sql_query('select * from '+table_name+';',db3)



,index,id,FIO_prep,subject,Lec_beg,Lec_end
0,0,1,Sergeev BB,Пульмонологическое отделение,2023-11-29 12:20:00,2023-12-29 13:55:00


что почитать по Pandas: https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html

#Доступ к базам данных MySQL

In [17]:
pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.0/34.0 MB 55.0 MB/s eta 0:00:00


In [18]:
from getpass import getpass
from mysql.connector import connect, Error

try:
    with connect(
        host="localhost",
        user=input("Имя пользователя: "),
        password=getpass("Пароль: "),
    ) as connection:
        create_db_query = "CREATE DATABASE test_1"
        with connection.cursor() as cursor:
            cursor.execute(create_db_query)
except Error as e:
    print(e)

Имя пользователя: root
Пароль: ··········
2002: Can't connect to local MySQL server through socket '%-.100s' (%s) (Warning: not all arguments converted during string formatting)


In [ ]:
#просмотр всех таблиц, хранящихся в базе данных
try:
    with connect(
        host="localhost",
        user=input("Введите имя пользователя: "),
        password=getpass("Введите пароль: "),
    ) as connection:
        show_db_query = "SHOW DATABASES"
        with connection.cursor() as cursor:
            cursor.execute(show_db_query)
            for db in cursor:
                print(db)
except Error as e:
    print(e)

Введите имя пользователя: root
Введите пароль: ··········
2003 (HY000): Can't connect to MySQL server on 'localhost:3306' (99)


In [ ]:
#подключение к существующей БД
try:
    with connect(
        host="localhost",
        user=input("Имя пользователя: "),
        password=getpass("Пароль: "),
        database="prakt_1",
    ) as connection:
        print(connection)
except Error as e:
    print(e)
sql_create = """CREATE TABLE students (id integer PRIMARY KEY, FIO text, DR text)"""

with connection.cursor() as cursor:
    cursor.execute(sql_create)
    connection.commit()


sql_show = "DESCRIBE students"
with connection.cursor() as cursor:
    cursor.execute(sql_show)
    res = cursor.fetchall()
    for row in res:
        print(row)

sql_insert="INSERT INTO students VALUES (1,'Иванов H.P.','2004'),(2,'Смирнов В.С.','2002')"

with connection.cursor() as cursor:
    cursor.execute(sql_insert)
    connection.commit()

sql_select = "SELECT * FROM students WHERE id <3 ORDER BY DR DESC"
with connection.cursor() as cursor:
    cursor.execute(sql_select)
    result = cursor.fetchall()
    for row in result:
        print(row)

In [ ]:
from getpass import getpass
from mysql.connector import connect, Error

old_id = input("Введите id студента: ")
new_fio = input("Введите обновлённые сведения по ФИО: ")
update_sql = """UPDATE students SET FIO = "%s" WHERE  id = "%s";
                SELECT * FROM students WHERE id = "%s"
             """ % (
    new_fio,
    old_id,
    old_id,
)

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
        database="test_1",
    ) as connection:
        with connection.cursor() as cursor:
            for result in cursor.execute(update_sql, multi=True):
                if result.with_rows:
                    print(result.fetchall())
            connection.commit()
except Error as e:
    print(e)

--- что почитать по MySQL на Python: 1) https://www.internet-technologies.ru/articles/posobie-po-mysql-na-python.html

2) https://metanit.com/python/database/

3) https://github.com/rasbt/python_reference/tree/master/tutorials/sqlite3_howto

--- по SQLite:    https://github.com/DataLatata/python-school/blob/master/41-sqlite-database-in-python/sqlite-and-pandas.ipynb

#Работа с СУБД PostgreSQL

In [19]:
pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 28.1 MB/s eta 0:00:00


In [20]:
import psycopg2
psycopg2.__libpq_version__

160000

In [26]:
# Подключение к серверу БД, просмотр информации о сервере PostgreSQL
import psycopg2
from psycopg2 import Error

try:
    connection = psycopg2.connect(user="postgres",
                                  # пароль, который указали при установке PostgreSQL
                                  password="*******",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="*****") #as cn: цith cn.cursor() as cur: ...

    # Курсор для выполнения операций с БД
    cur = connection.cursor()
    # Распечатать сведения о PostgreSQL
    print("Информация о сервере PostgreSQL")
    print(connection.get_dsn_parameters(), "\n")
    # Выполнение SQL-запроса
    cur.execute("SELECT version();")
    # Получить результат
    record = cur.fetchone()
    print("Вы подключены к - ", record, "\n")

except (Exception, Error) as error:
    print("Ошибка при работе с PostgreSQL", error)
finally:
    if connection:
        cur.close()
        connection.close()
        print("Соединение с PostgreSQL закрыто")

Ошибка при работе с PostgreSQL connection to server at "127.0.0.1", port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?



NameError: name 'connection' is not defined

In [ ]:
# Подключение к существующей БД test_db
import psycopg2
from psycopg2 import Error
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
try:
    # Подключение к существующей базе данных
    connection = psycopg2.connect(user="postgres",
                                  # пароль, который указали при установке PostgreSQL
                                  password="******",
                                  host="127.0.0.1",
                                  port="5432")
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    # Курсор для выполнения операций с базой данных
    cursor = connection.cursor()
    sql_create_database = 'create database test_db'
    cursor.execute(sql_create_database)
except (Exception, Error) as error:
    print("Ошибка при работе с PostgreSQL", error)
finally:
    if connection:
        cursor.close()
        connection.close()
        print("Соединение с PostgreSQL закрыто")

In [ ]:
import psycopg2
conn = psycopg2.connect(dbname='testdb',
                        user='postgres',
                        password='11u424',
                        host='localhost' ,
                        port=5432)
cur = conn.cursor()
cur.execute("CREATE TABLE students (id integer PRIMARY KEY, FIO text, DR text)")
cur.execute("INSERT INTO students VALUES (1,'Иванов H.P.','2004'),(2,'Смирнов В.С.','2002')",("postgres", "123"))
cur.close()
conn.commit()

In [ ]:
with conn.cursor() as cursor:
    conn.autocommit = True
    values = [
        (12, 'Васильев А.Ф.', '2003'),
        (13, 'Долгин Ч.З.', '2001'),
        (14, 'Федорова Ц.З.', '2002'),
    ]
    insert = sql.SQL('INSERT INTO students (id, FIO, DR) VALUES {}').format(sql.SQL(',').join(map(sql.Literal, values)))
    cursor.execute(insert)

In [ ]:
cur.execute("SELECT id, fio, dr FROM students")
cur.fetchall()

In [ ]:
cur.execute('UPDATE students SET fio = %(fio)s WHERE id = %(id)s', {'fio':'Петров М.Л.', 'id':'789'})
conn.commit()

In [ ]:
cur.close()
conn.close()